In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

In [11]:
import json
data = []
with open('/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        if features["context"]:
            continue
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

data = data[:2000]

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Inference before fine tuning


In [12]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe.
2. Take a trip to Europe.
3. Take a trip to Europe.
4. Take a trip to Europe.
5. Take a trip to Europe.
6. Take a trip to Europe.
7. Take a trip to Europe.
8. Take a trip to Europe.
9. Take a trip to Europe.
10. Take a trip to Europe.
11. Take a trip to Europe.
12. Take a trip to Europe.
13. Take a trip to Europe.
14. Take a trip to Europe.
15. Take a trip to Europe.
16. Take a trip to Europe.
17. Take a trip to Europe.
18. Take a trip to Europe.
19. Take a trip to Europe.
20. Take a trip to Europe.
21. Take a trip to Europe.
22. Take a trip to Europe.
23. Take a trip to Europe.
24. Take a trip to Europe.
25. Take a trip to Europe.
26. Take a trip to Europe.
27. Take a trip to Europe.
28. Take a trip to Europe.
29. Take a trip to Europe.
30. Take a trip to Europe.
31. Take a trip to Europe.
32. Take a trip to Europe.
33. Take a trip to Europe.
34. Take a trip to Europe.
35. Take a trip to Europe.
36

In [ ]:
# gemma_lm.load_weights("/kaggle/working/gemma_weights.weights.h5")

In [10]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make glucose. The glucose is then used by the plant to make energy and grow.

Explanation:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make gluc

In [13]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [14]:
gemma_lm.preprocessor.sequence_length = 512
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1457s 719ms/step - loss: 0.4398 - sparse_categorical_accuracy: 0.5369
Epoch 2/2
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1439s 715ms/step - loss: 0.4151 - sparse_categorical_accuracy: 0.5571


In [6]:
import os
os.remove("/kaggle/working/gemma_weights.weights.h5")

In [ ]:
# gemma_lm.save("/kaggle/working/gemma_lm_model.keras")
gemma_lm.save_weights("/kaggle/working/gemma_weights.weights.h5")

In [ ]:
# !pip install python-doctr[tf] 
# !pip install tf2onnx 

# !pip install tensorflow==2.15.1

In [20]:
import pandas as pd
df = pd.read_json('/kaggle/input/meddialogue/english-train.json')
df.head() 

,description,utterances
0,throat a bit sore and want to get a good imune...,[patient: throat a bit sore and want to get a ...
1,"hey there i have had cold ""symptoms"" for over ...","[patient: hey there i have had cold ""symptoms""..."
2,i have a tight and painful chest with a dry co...,[patient: i have a tight and painful chest wit...
3,what will happen after the incubation period f...,[patient: what will happen after the incubatio...
4,suggest treatment for pneumonia,[patient: just found out i was pregnant. yeste...


In [ ]:
import numpy as np 
import pandas as pd
import re

In [ ]:
data = []
txt_files = [
    '/kaggle/input/meddialogue/healthcaremagic_dialogue_1.txt',
    '/kaggle/input/meddialogue/healthcaremagic_dialogue_2.txt',
    '/kaggle/input/meddialogue/healthcaremagic_dialogue_3.txt',
    '/kaggle/input/meddialogue/healthcaremagic_dialogue_4.txt'
]
for file_path in txt_files:
    txt_file = open(file_path, 'r')
    txt = txt_file.read()
    for line in txt[3:].split('\nid='):
        sections = line.split('\n')
        dialog = '\n'.join(sections[7:])
        turns = [i for i in re.split('Doctor:\n|Patient:\n', dialog) if i]                     
        row = [sections[0], sections[1], sections[4], len(turns), dialog]
        data.append(row)

len(data) 

In [28]:
df = pd.DataFrame(data, columns=['id', 'url', 'description', 'turns', 'dialogue'])
df["dialogue"][1]



NameError: name 'pd' is not defined

In [ ]:
def extract_dialogue(dialogue):
    patient_text = dialogue.split("Doctor:")[0].replace("Patient:\n", "").strip()
    doctor_text = dialogue.split("Doctor:")[1].replace("Doctor:\n", "").strip()
    return patient_text, doctor_text

combined_dialogues = []

for index, row in df.iterrows():
    patient_text, doctor_text = extract_dialogue(row["dialogue"])
    combined_text = f"Instruction:\n{patient_text}\n\nResponse:\n{doctor_text}"
    combined_dialogues.append(combined_text)

In [23]:
combined_dialogues[0]

mednewData = combined_dialogues[:4000]

NameError: name 'combined_dialogues' is not defined

In [ ]:
# gemma_lm.preprocessor.sequence_length = 512
# optimizer = keras.optimizers.AdamW(
#     learning_rate=5e-5,
#     weight_decay=0.01,
# )
# optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# gemma_lm.compile(
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer=optimizer,
#     weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
# )
# gemma_lm.fit(mednewData, epochs=1, batch_size=1)

In [26]:
gemma_lm.save_weights("/kaggle/working/gemma_weights.weights.h5")

In [26]:
!pip install pytesseract

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [15]:
!pip install Pillow
from PIL import Image 
import pytesseract

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [18]:
def ocr_pytesseract(image_path):
    image = Image.open(image_path)  # Open the image
    text = pytesseract.image_to_string(image)  # Extract text
    return text



In [17]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
There are so many things to do in Europe, it's hard to know where to start. Here are some ideas to get you started:

1. Visit the Eiffel Tower in Paris
2. Take a river cruise on the Danube in Vienna
3. Explore the Colosseum in Rome
4. Visit the Vatican in Rome
5. Take a train ride through the Swiss Alps
6. Visit the Louvre in Paris
7. Take a cooking class in Florence
8. Visit the Acropolis in Athens
9. Take a day trip to Pompeii
10. Visit the Colosseum in Rome


In [19]:
image_path = '/kaggle/input/testpresimage/image.png'
presText = ocr_pytesseract(image_path)
print(presText)

Pr. A. Sharma, New Delhi Clinic, +91 9876543210

Today, a middle-aged man, Kumar, came in complaining of dizziness and fatigue. He’s been
having issues with high blood pressure for a while, and after a quick check-up, it turns out he
also has blood sugar levels that aren't quite right. We've talked about starting him on some
medication to keep both conditions under control.

For his heart, he'll need one pill in the morning — something like Amlodipine, 5 mg. For his
sugar, it’s a twice-daily thing after meals — maybe Metformin, 500 mg. And at night, there’s
that statin — Atorvastatin, 10 mg should do the trick.

| reminded him about cutting down on salt, sticking to whole grains, and squeezing in at least
half an hour of light exercise each day. He’s supposed to check back in with me in a month,
but if things go south, he knows to call sooner.

— A. Sharma (signed)
20/10/2024



In [22]:
presTemp = """
Please analyze the following medical prescription and extract the following details:

Chief Complaints: Describe the patient's main complaints, including symptoms, duration, and type of seizures.

Patient Details:

Name: Full name of the patient
Date of Birth: Date of birth in MM/DD/YYYY format
Gender: Gender as Male/Female/Other
Age: Age in years
Address: Patient's current address
Past Medical History:

Birth History: Details of birth, such as complications or full-term/premature
Developmental Milestones: Brief on developmental milestones met or delayed
Past Illnesses: List any significant past medical events like febrile seizures
Seizure Details:

Type of Seizure: Specific type of seizure (e.g., focal, generalized)
Frequency: Number of times seizures occur (per day, week, etc.)
Duration: Typical duration of seizures (seconds/minutes)
Postictal State: Symptoms after a seizure, such as confusion or fatigue
Comorbidities and Family History:

Family History of Epilepsy: Indicate Yes or No
Family History of Febrile Seizures: Indicate Yes or No
Other Conditions: Any other family history or comorbidities
Current Treatment:

Medications: List current medications, including name, dosage, and frequency
Adverse Effects: List any side effects reported
Physical Examination Findings:

General Appearance: Description of the patient's general physical condition
Neurological Exam: Any findings from neurological assessments
Investigation Results:

EEG Findings: Summarize any findings from EEG
VEEG Findings: Summarize findings from VEEG, if applicable
CT/MRI Results: Include any findings from imaging studies like CT or MRI
Medical Prescription Text: 
{presText}

"""

prestFormattedTem = presTemp.format(presText=presText)
print(prestFormattedTem) 


Please analyze the following medical prescription and extract the following details:

Chief Complaints: Describe the patient's main complaints, including symptoms, duration, and type of seizures.

Patient Details:

Name: Full name of the patient
Date of Birth: Date of birth in MM/DD/YYYY format
Gender: Gender as Male/Female/Other
Age: Age in years
Address: Patient's current address
Past Medical History:

Birth History: Details of birth, such as complications or full-term/premature
Developmental Milestones: Brief on developmental milestones met or delayed
Past Illnesses: List any significant past medical events like febrile seizures
Seizure Details:

Type of Seizure: Specific type of seizure (e.g., focal, generalized)
Frequency: Number of times seizures occur (per day, week, etc.)
Duration: Typical duration of seizures (seconds/minutes)
Postictal State: Symptoms after a seizure, such as confusion or fatigue
Comorbidities and Family History:

Family History of Epilepsy: Indicate Yes or 

In [25]:
prompt = template.format(
    instruction=prestFormattedTem,
    response="", )

# print(prompt)


generated_text = gemma_lm.generate(prompt,  max_length=900)

print(generated_text)

Instruction:

Please analyze the following medical prescription and extract the following details:

Chief Complaints: Describe the patient's main complaints, including symptoms, duration, and type of seizures.

Patient Details:

Name: Full name of the patient
Date of Birth: Date of birth in MM/DD/YYYY format
Gender: Gender as Male/Female/Other
Age: Age in years
Address: Patient's current address
Past Medical History:

Birth History: Details of birth, such as complications or full-term/premature
Developmental Milestones: Brief on developmental milestones met or delayed
Past Illnesses: List any significant past medical events like febrile seizures
Seizure Details:

Type of Seizure: Specific type of seizure (e.g., focal, generalized)
Frequency: Number of times seizures occur (per day, week, etc.)
Duration: Typical duration of seizures (seconds/minutes)
Postictal State: Symptoms after a seizure, such as confusion or fatigue
Comorbidities and Family History:

Family History of Epilepsy: Ind

In [ ]:
def parse_prescription_data(model_output):
    lines = model_output.strip().split('\n')
    
    prescription_data = {}
    
    current_section = None
    
    for line in lines:
        line = line.strip()
        if line.endswith(':'):
            current_section = line[:-1] 
        elif current_section:
            prescription_data[current_section] = prescription_data.get(current_section, '') + ' ' + line
    
    for key in prescription_data:
        prescription_data[key] = prescription_data[key].strip()
    
    return prescription_data

In [ ]:
model_response = generated_text.split("Response:")[1]
if model_response.find("Medical Prescription Text"):
    model_response = model_response.split("Medical Prescription Text")[0]

In [ ]:
import pandas as pd

parsed_data = parse_prescription_data(model_response)

df_prescription = pd.DataFrame([parsed_data]).drop('Please analyze the following medical prescription and extract the following details', axis=1)
df_prescription

In [ ]:
# import csv
# import os

# response_part = generated_text.split("Response:")[1].strip()

# lines = response_part.split("\n")

# data = {
#     "Patient Name": "",
#     "Patient Age": "",
#     "Patient Gender": "",
#     "Date of Prescription": "",
#     "Diagnosis": "",
#     "Medications": "",
#     "Additional Instructions": "",
#     "Doctor's Name": ""
# }

# medications = []
# additional_instructions = []

# for line in lines:
#     if line.startswith("Patient Name:"):
#         data["Patient Name"] = line.split(": ", 1)[1]
#     elif line.startswith("Patient Age:"):
#         data["Patient Age"] = line.split(": ", 1)[1]
#     elif line.startswith("Patient Gender:"):
#         data["Patient Gender"] = line.split(": ", 1)[1]
#     elif line.startswith("Date of Prescription:"):
#         data["Date of Prescription"] = line.split(": ", 1)[1]
#     elif line.startswith("Diagnosis:"):
#         data["Diagnosis"] = line.split(": ", 1)[1]
#     elif line.startswith("Medications:"):
#         continue  # Skip "Medications" label
#     elif line.startswith("Additional Instructions:"):
#         continue  # Skip "Additional Instructions" label
#     elif line.startswith("Doctor's Name:"):
#         data["Doctor's Name"] = line.split(": ", 1)[1]
#     elif line.startswith("1.") or line.startswith("2.") or line.startswith("3.") and "mg" in line:
#         medications.append(line.strip())
#     elif line.startswith("1.") or line.startswith("2.") or line.startswith("3."):
#         additional_instructions.append(line.strip())

# data["Medications"] = "; ".join(medications)
# data["Additional Instructions"] = "; ".join(additional_instructions)

# file_exists = os.path.isfile('prescription_extracted.csv')


# print(data)

In [ ]:
csv_file_path = 'prescription_data.csv' 
df_prescription.to_csv(csv_file_path, mode='a', index=False, header=not os.path.exists(csv_file_path))

print(f"Data saved to {csv_file_path}")

print("Data appended to 'prescription_extracted.csv'")